In [1]:
import numpy as np
import ROOT
import os
import csv

Welcome to JupyROOT 6.28/04


# Main analysis
To do:
- Calculate the transverse muon and proton energies, draw a plot of that
- Upon requiring that there is only one proton and the transverse energy is low, <200 MeV or so, draw a plot of the muon and proton energies, and hopefully we should see a spike around 236 MeV?

In [82]:
def genTrackTimeOffset(source_dir, output_filename, scan_start, hist_bins, nbeams = 5, hist_max = 250):
    # Creating TChain from all files in source directory
    Chain = ROOT.TChain( "MasterAnaDev", "Chain" )
    nfiles = 0 

    for (root, dirs, files) in os.walk(f"{source_dir}/"):
        for file in files:
            if ("MasterAnaDev_data_AnaTuple_run" in file) and ("Playlist.root" in file):
                Chain.Add(f"{root}/{file}")
                nfiles += 1

    print(f"Chain created with {Chain.GetEntries()} entries from {nfiles} files.")

    # Setting only the relevant branches as active
    Chain.SetBranchStatus("*", False)
    Chain.SetBranchStatus("ev_gate", True)
    Chain.SetBranchStatus("vtx", True)
    Chain.SetBranchStatus("muon_trackVertexTime", True)
    
    # Histogram to which we can fit the beam structure
    tracktime_hist = ROOT.TH1D( "Track_Time_Hist", ";Track Time (ns)", hist_bins, 0, hist_max )

    # Not sure what these are for...
    prev_gate = 0
    prev_time = 0.

    # Calculating track time for each event
    for entry in Chain:        
        vtx_time = entry.muon_trackVertexTime - (entry.vtx[2] - 4000)/300.
        if entry.ev_gate == prev_gate:
            tracktime = vtx_time - prev_time
            if tracktime < hist_max:
                tracktime_hist.Fill(tracktime)

        prev_gate = entry.ev_gate
        prev_time = vtx_time

    bin_density = hist_bins/hist_max # bins per time

    fit_radius  = 6                              # radius about the mean beam time in ns to fit a gaussian curve
    beam_radius = round(fit_radius*bin_density)  # radius aobut the mean beam time in bins
    
    beam_period      = 18.831                          # beam periodicity in ns
    beam_period_bins = round(beam_period*bin_density)  # beam periodicity in bins  
    
    curr_time  = scan_start  # current time for the search

    beam_times = []
    start_times = []

    for beam_number in range(nbeams):
        fit = tracktime_hist.Fit("gaus", "S Quiet", "", curr_time-fit_radius, curr_time+fit_radius)
        time = fit.Parameter(1)
        start_time = time - (beam_number)*beam_period
        beam_times.append(time)
        start_times.append(start_time)

        curr_time += beam_period 

    mean_start_time = np.mean(start_times)
    
    output_file = open(f"{output_filename}_step.txt", 'w')
    output_file.write(f"{mean_start_time}")
    output_file.close()

In [83]:
def ImportOffset(filename):
    file = open(f"{filename}.txt",'r')
    csvfile = csv.reader(file)
    for ttoffset in csvfile:
        return float(ttoffset[0])

In [84]:
def ParallelizeFiles(source_dir, output_filename, nsets):
    # Putting all files to process into a single list
    all_files = []

    for (root, dirs, files) in os.walk(f"{source_dir}/"):
        for file in files:
            if ("MasterAnaDev_data_AnaTuple_run" in file) and ("Playlist.root" in file):
                all_files.append(f"{root}/{file}")

    # Splitting the files into n equal sets
    file_sets = np.array_split(all_files, nsets)
    print(f"{nsets} file sets with length {len(file_sets[0])} created from {len(all_files)} files")

    # Writing file sets into an output file
    output_file = open(f"{output_filename}.txt", "w", newline = '')
    csvfile = csv.writer(output_file)
    
    for file_set in file_sets:
        csvfile.writerow(file_set)
    output_file.close()

# Function to get specific file set from a pre-generated list of file sets
def GetFileSet(fileset_name, set_number):
    input_file = open(f"{fileset_name}.txt", newline = '')
    csvfile = csv.reader(input_file)
    for i, row in enumerate(csvfile):
        if i == set_number:
            return row

In [85]:
def CreateDataTree(fileset_name, set_number):
    # Creating a TChain for all the files in the analysis
    Chain = ROOT.TChain( "MasterAnaDev", "Chain" )

    # Equivalent to the GetFileSet function
    input_file = open(f"{fileset_name}.txt", newline = '')
    csvfile = csv.reader(input_file)
    for i, row in enumerate(csvfile):
        if i == set_number:
            file_set = row

    # Looping through all the files in the source directory and any subfolders
    for file in file_set:
        Chain.AddFile(file)

    # To speed up processing, inactivating irrelevant branches
    Chain.SetBranchStatus("*", False)

    branch_list = {'ev_gate',
                   'ev_subrun',
                   'ev_run',
                   'vtx',
                   'muon_trackVertexTime', 
                   'MasterAnaDev_muon_E',
                   'MasterAnaDev_muon_P',
                   'MasterAnaDev_muon_Px',
                   'MasterAnaDev_muon_Py',
                   'MasterAnaDev_muon_Pz',
                   'MasterAnaDev_muon_theta', 
                   'MasterAnaDev_minos_trk_is_ok',
                   'MasterAnaDev_proton_E_fromdEdx',
                   'MasterAnaDev_proton_T_fromdEdx',
                   'MasterAnaDev_proton_P_fromdEdx',
                   'MasterAnaDev_proton_Px_fromdEdx',
                   'MasterAnaDev_proton_Py_fromdEdx',
                   'MasterAnaDev_proton_Pz_fromdEdx',
                   # 'MasterAnaDev_proton_theta_fromdEdX',
                   # 'n_minos_matches', 
                   # 'MasterAnaDev_in_fiducial_area',
                   'MasterAnaDev_pion_E',
                   'MasterAnaDev_pion_P',
                   'MasterAnaDev_pion_theta',  
                   # 'recoil_energy_nonmuon_vtx200mm'
                  }

    # Reactivating relevant branches
    for branch in branch_list:
        Chain.SetBranchStatus(branch, True)

    # Copying active data into a new file so it's easier to manipulate
    DataTree = Chain.CloneTree(0)
    DataTree.CopyEntries(Chain)
    
    return DataTree

In [86]:
def getLTMomenta(vector1, vector2):
    beam_correction = ROOT.Math.RotationX(-0.0575958653)

    v1_corr = beam_correction(vector1)
    v2_corr = beam_correction(vector2)

    v_combo = v1_corr + v2_corr
    
    v_transverse = np.sqrt( (v_combo.X())**2 + (v_combo.Y())**2 )
    v_longitude  = v_combo.Z()

    return v_transverse, v_longitude

In [93]:
def FileAnalysis(output_filename, fileset_name, set_number, beam_start_time):
    print(f"Beginning Analysis on file set {set_number}")
    print("--------------------------------------------")
    # Importing Data
    output_file = ROOT.TFile(f"{output_filename}{set_number}.root", 'recreate')
    data_tree = CreateDataTree(fileset_name, set_number)
    print("Data tree created")

    """ Defining Analysis Branches """
    "=========================================================="
    ana_tree = data_tree.CloneTree(0)
    
    track_time = np.array([0.]) # event track time (ns)
    beam_offset = np.array([0.]) # event offset from beam time (ns)
    transferred_mom = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) # transferred 4mom |t| in muon-pion interactions [MeV^2]
    muon_proton_P_L = np.array([0.])
    muon_proton_P_T = np.array([0.])
    npions = np.array([0])
    
    ana_tree.Branch("track_time", track_time, "tracktime/D")
    ana_tree.Branch("beam_offset", beam_offset, "beam_offset/D")
    ana_tree.Branch("transferred_mom", transferred_mom, "transferred_mom/D")
    ana_tree.Branch("muon_proton_P_L", muon_proton_P_L, "muon_proton_P_L/D")
    ana_tree.Branch("muon_proton_P_T", muon_proton_P_T, "muon_proton_P_T/D")
    ana_tree.Branch("npions", npions, "npions")
    
    print("Analysis branches created")
    "=========================================================="

    """ Setting Branch Addresses """
    "=========================================================="
    ev_run    = np.array([0.])
    ev_subrun = np.array([0.])
    ev_gate   = np.array([0.])
    data_tree.SetBranchAddress("ev_run", ev_run)
    data_tree.SetBranchAddress("ev_subrun", ev_subrun)
    data_tree.SetBranchAddress("ev_gate", ev_gate)

    vtx         = np.zeros(4)
    minos_match = np.array([0.])
    muonTVT     = np.array([0.])
    # nparticles  = np.array([0] )
    data_tree.SetBranchAddress('vtx', vtx)
    data_tree.SetBranchAddress('MasterAnaDev_minos_trk_is_ok', minos_match)
    data_tree.SetBranchAddress('muon_trackVertexTime', muonTVT)
    # data_tree.SetBranchAddress('VertexTrackMultiplicity', nparticles)
    
    muonE     = np.array([0.])
    muonP     = np.array([0.])
    muonPx    = np.array([0.])
    muonPy    = np.array([0.])
    muonPz    = np.array([0.])
    muonTheta = np.array([0.])
    data_tree.SetBranchAddress('MasterAnaDev_muon_E', muonE)
    data_tree.SetBranchAddress('MasterAnaDev_muon_P', muonP)
    data_tree.SetBranchAddress('MasterAnaDev_muon_Px', muonPx)
    data_tree.SetBranchAddress('MasterAnaDev_muon_Py', muonPy)
    data_tree.SetBranchAddress('MasterAnaDev_muon_Pz', muonPz)
    data_tree.SetBranchAddress('MasterAnaDev_muon_theta', muonTheta)

    pionE     = np.zeros(10)
    pionP     = np.zeros(10)
    pionTheta = np.zeros(10)
    data_tree.SetBranchAddress('MasterAnaDev_pion_E', pionE)
    data_tree.SetBranchAddress('MasterAnaDev_pion_P', pionP)
    data_tree.SetBranchAddress('MasterAnaDev_pion_theta', pionTheta)

    protonT     = np.array([0.])
    protonP     = np.array([0.])
    protonPx    = np.array([0.])
    protonPy    = np.array([0.])
    protonPz    = np.array([0.])
    protonTheta = np.array([0.])
    data_tree.SetBranchAddress('MasterAnaDev_proton_T_fromdEdx', protonT)
    data_tree.SetBranchAddress('MasterAnaDev_proton_P_fromdEdx', protonP)
    data_tree.SetBranchAddress('MasterAnaDev_proton_Px_fromdEdx', protonPx)
    data_tree.SetBranchAddress('MasterAnaDev_proton_Py_fromdEdx', protonPy)
    data_tree.SetBranchAddress('MasterAnaDev_proton_Pz_fromdEdx', protonPz)
    print("Branch addresses set")
    "=========================================================="

    """ Analysis Loop """
    "=========================================================="
    beam_period = 18.831
    beam_size = beam_period/2 + 0.00001 # Adding a very small overlap to avoid the possibility of getting stuck in the beam offset loop, better way to prevent this? 

    N = data_tree.GetEntries()
    n = 0
    # Looping through all entries
    while n < N-1:
        # Checking for entries that have a match in run, subrun, and gate numbers
        data_tree.GetEntry(n+1)
        run1, subrun1, gate1 = ev_run[0], ev_subrun[0], ev_gate[0]
        time1 = muonTVT[0] - (vtx[2] - 4000)/300
        data_tree.GetEntry(n)
        run0, subrun0, gate0 = ev_run[0], ev_subrun[0], ev_gate[0]
        time0 = muonTVT[0] - (vtx[2] - 4000)/300
        """ Properties """
        # Script runs a bit differently depending on whether or not there is an gate, run subrun match
        if (run1 == run0) and (subrun1 == subrun0) and (gate1 == gate0):
            
            """ Both Events - Track Time and Beam Offset """
            "------------------------------------------------------"
            trk_time = np.abs(time0-time1)
            
            if trk_time < beam_start_time - beam_size: # Flagging any events before the start time (-1111)
                track_time[0]  = -1111
                beam_offset[0] = -1111
            elif trk_time > 344 and trk_time < 352: # Flagging the strange peak at ~348 ns (-4444)
                track_time[0]  = -4444
                beam_offset[0] = -4444
            else:
                track_time[0] = trk_time
                offset_time = trk_time - beam_start_time # Adjusting for the beam start time
                
                while offset_time > beam_size: # Finding the beam offset by subtracting intervals of the beam period
                    offset_time -= beam_period
                    
                beam_offset[0] = offset_time
            "------------------------------------------------------"

            """ First Event - Transferred 4 Momentum """
            "------------------------------------------------------"
            if minos_match == 1:
                muonP_T = muonP * np.sin(muonTheta)
                muonP_L = muonP * np.cos(muonTheta)

                for i, piE in enumerate(pionE):
                    if piE < 0: # Flagging invalid pions in array (-8888)
                        transferred_mom[i] = -8888
                    else:
                        npions[0] += 1
                        piP     = pionP[i]
                        piTheta = pionTheta[i]
                    
                        pionP_T = piP * np.sin(piTheta)
                        pionP_L = piP * np.sin(piTheta)

                        transferred_mom[i] = (piE + muonE - pionP_L - muonP_L)**2 + (pionP_T + muonP_T)**2
            else:
                for i, piE in enumerate(pionE):
                    if piE > 0:
                        npions[0] += 1
                    transferred_mom[i] = -7777 # Flagging non matched events (-7777)
            "------------------------------------------------------"

            """ First Event - Proton-Muon Transverse Momentum """
            "------------------------------------------------------"
            if (muonP != 0) and (protonP != 0):
                muonP_vec = ROOT.Math.XYZVector(muonPx, muonPy, muonPz)
                protonP_vec = ROOT.Math.XYZVector(protonPx, protonPy, protonPz)

                P_T, P_L = getLTMomenta(muonP_vec, protonP_vec)
                muon_proton_P_T[0] = P_T
                muon_proton_P_L[0] = P_L
            else:
                muon_proton_P_T[0] = -8888
                muon_proton_P_L[0] = -8888
            "------------------------------------------------------"

            """ Filling First Event """
            "------------------------------------------------------"
            ana_tree.Fill()
            "------------------------------------------------------"

            """ Second Event - Transferred 4 Momentum """
            "------------------------------------------------------"
            data_tree.GetEntry(n+1) # Setting the current entry to the second event
            if minos_match == 1:
                muonP_T = muonP * np.sin(muonTheta)
                muonP_L = muonP * np.cos(muonTheta)

                for i, piE in enumerate(pionE):
                    if piE < 0: # Flagging invalid pions in array (-8888)
                        transferred_mom[i] = -8888
                    else:
                        npions[0] += 1
                        piP     = pionP[i]
                        piTheta = pionTheta[i]
                    
                        pionP_T = piP * np.sin(piTheta)
                        pionP_L = piP * np.sin(piTheta)

                        transferred_mom[i] = (piE + muonE - pionP_L - muonP_L)**2 + (pionP_T + muonP_T)**2
            else:
                for i, piE in enumerate(pionE):
                    if piE > 0:
                        npions[0] += 1
                    transferred_mom[i] = -7777 # Flagging non matched events (-7777)
            "------------------------------------------------------"

            """ Second Event - Proton-Muon Transverse Momentum """
            "------------------------------------------------------"
            muonP_vec = ROOT.Math.XYZVector(muonPx, muonPy, muonPz)
            protonP_vec = ROOT.Math.XYZVector(protonPx, protonPy, protonPz)

            P_T, P_L = getLTMomenta(muonP_vec, protonP_vec)
            muon_proton_P_T[0] = P_T
            muon_proton_P_L[0] = P_L
            "------------------------------------------------------"

            """ Filling Second Event """
            "------------------------------------------------------"
            ana_tree.Fill()
            "------------------------------------------------------"
            n += 2
        
        else:
            track_time[0] = -9999
            beam_offset[0] = -9999
            for i in range(len(transferred_mom)):
                transferred_mom[i] = -9999
            muon_proton_P_T[0] = -9999
            muon_proton_P_L[0] = -9999
            npions[0] = -9999

            ana_tree.Fill()
            n += 1
        "------------------------------------------------------"
    "=========================================================="
    
    output_file.WriteObject(ana_tree, "Analysis")
    output_file.Close()

In [95]:
def CondenseAnalysis(output_filename):
    AnaChain = ROOT.TChain("Analysis", "AnaChain")
    
    for (root, dirs, files) in os.walk("./"):
        for file in files:
            if output_filename in file:
                AnaChain.Add(f"{root}/{file}")

    output_file = ROOT.TFile(f"{output_filename}_cond.root", 'recreate')
    
    output_tree = AnaChain.CloneTree(0)
    output_tree.CopyEntries(AnaChain)
    output_tree.SetName("Analysis")
    output_tree.SetTitle("Minerva Analysis")
    
    """ Analysis Histograms """
    "=========================================================="
    track_time_hstack = ROOT.THStack("TrackTimeHist", ";Track Time [ns]")
    track_time_hin = ROOT.TH1D("TrackTimeHist_in", ";Track Time [ns]", 10000, 0., 10000.)
    track_time_hout = ROOT.TH1D("TrackTimeHist_out", ";Track Time [ns]", 10000, 0., 10000.)
    track_time_hin.SetLineColor(1)
    track_time_hout.SetLineColor(2)

    beam_period = 18.831
    beam_size = beam_period/2 + 0.00001
    
    offset_hstack = ROOT.THStack("OffsetHist", ";Beam Offset [ns]")
    offset_hin = ROOT.TH1D( "OffsetHist_in", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_hout = ROOT.TH1D( "OffsetHist_out", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_htotal = ROOT.TH1D( "OffsetHist_Total", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_hin.SetLineColor(1)
    offset_hout.SetLineColor(2)
    
    print("Histograms Created")
    "=========================================================="

    """ Filling Histograms """
    "=========================================================="
    for event in AnaChain:
        beam_offset = event.beam_offset
        offset_htotal.Fill(beam_offset)
        if np.abs(beam_offset) < 5:
            track_time_hout.Fill(event.track_time)
            offset_hout.Fill(beam_offset)
        elif np.abs(beam_offset) >= 5 and np.abs(beam_offset) < 100:
            track_time_hin.Fill(event.track_time)
            offset_hin.Fill(beam_offset)
    "=========================================================="

    """ Histogram Drawing """
    "=========================================================="
    # Track Time
    track_time_hstack.Add(track_time_hout)
    track_time_hstack.Add(track_time_hin)
    "-----------------------------"
    
    """ Beam Offset """
    "-----------------------------"
    offset_hstack.Add(offset_hout)
    offset_hstack.Add(offset_hin)
    
    oCanvas = ROOT.TCanvas("BeamOffsetHist", "BeamOffsetHist")
    oCanvas.cd()
    offset_hstack.Draw()
    oCanvas.Update()
    
    # Defining function for a gaussian on top of a uniform background
    gaus_bg = ROOT.TF1("gaus_bg", "gaus(0) + [3]", -9.5,9.5) 
    gaus_bg.SetParName(3, "background")
    
    gaus_bg.SetParameters(2000, 0, 2, 20)

    # Fitting Packet Offset 
    hist_fit = offset_htotal.Fit("gaus_bg", "S Quiet", "", -9.5, 9.5)
    hist_fit.Draw()
    oCanvas.Update()
    "-----------------------------"

    output_file.WriteObject(output_tree, output_tree.GetName())
    output_file.WriteObject(track_time_hstack, "TrackTimeHist")
    output_file.WriteObject(oCanvas, "BeamOffsetHist")
    output_file.Close()

In [89]:
def RunAnalysis(output_filename, file_set, set_numbers, tracktime_file, beam_time_cut = 5):
    beam_start_time = ImportOffset(tracktime_file)
    for set_number in set_numbers:
        FileAnalysis(output_filename, file_set, set_number, beam_start_time, beam_time_cut)

In [90]:
def CreateMuonHists(filename, time_cut, ETheta_rez = (50,50), E_rez = 100, plot_individual = True):
    file = ROOT.TFile(f"{filename}.root", 'update')
    tree = file.Get("Analysis")
    
    tree.SetBranchStatus("*", False)
    branches = ("beam_offset", "MasterAnaDev_muon_E", "MasterAnaDev_muon_theta", "MasterAnaDev_minos_trk_is_ok")
    for branch in branches:
        tree.SetBranchStatus(branch, True)
        
    """ Creating 2D Histograms """
    "=========================================================="
    E_resolution = ETheta_rez[0]
    Theta_resolution = ETheta_rez[1]
    
    # ETheta_in_nomatch = ROOT.TH2D("In Time, No MINOS Match", ";Muon Energy [MeV] vs Angle", E_resolution, 0, 10000, Theta_resolution, 0, 2 )
    # ETheta_out_nomatch = ROOT.TH2D("Out Time, No MINOS Match", ";Muon Energy [MeV] vs Angle", E_resolution, 0, 10000, Theta_resolution, 0, 2 )

    # ETheta_in_match = ROOT.TH2D("In Time, MINOS Match", ";Muon Energy [MeV] vs Angle", E_resolution, 0, 10000, Theta_resolution, 0, 2 )
    # ETheta_out_match = ROOT.TH2D("Out Time, MINOS Match", ";Muon Energy [MeV] vs Angle", E_resolution, 0, 10000, Theta_resolution, 0, 2 )
    
    # ETheta_in = ROOT.TH2D("In Time, All", ";Muon Energy [MeV] vs Angle", E_resolution, 0, 10000, Theta_resolution, 0, 2 )
    # ETheta_out = ROOT.TH2D("Out Time, All", ";Muon Energy [MeV] vs Angle", E_resolution, 0, 10000, Theta_resolution, 0, 2 )
    "=========================================================="

    """ Creating 1D Histograms """
    "=========================================================="
    E_in_nomatch  = ROOT.TH1D("In Time, No MINOS Match", ";Muon Energy [MeV]", E_rez, 0, 1000)
    E_in_nomatch.SetLineColor(1)
    E_out_nomatch = ROOT.TH1D("Out TIme, No MINOS Match", ";Muon Energy [MeV]", E_rez, 0, 1000)
    E_out_nomatch.SetLineColor(2)

    E_in_match = ROOT.TH1D("In Time, MINOS Match", ";Muon Energy [MeV]", E_rez, 0, 1000)
    E_in_match.SetLineColor(1)
    E_out_match = ROOT.TH1D("Out Time, MINOS Match", ";Muon Energy [MeV]", E_rez, 0, 1000)
    E_out_match.SetLineColor(2)

    E_in = ROOT.TH1D("In Time, All", ";Muon Energy [MeV]", E_rez, 0, 1000)
    E_in.SetLineColor(1)
    E_out = ROOT.TH1D("Out Time, All", ";Muon Energy [MeV]", E_rez, 0, 1000)
    E_out.SetLineColor(2)
    "=========================================================="

    """ Event Loop """
    "=========================================================="
    for event in tree:
        if event.beam_offset < -100:
            continue
        beam_timing = np.abs(event.beam_offset)
        muonE = event.MasterAnaDev_muon_E
        muonTheta = event.MasterAnaDev_muon_theta
        minos_match = event.MasterAnaDev_minos_trk_is_ok
        """ Filling Non Minos Matched Histograms """
        "-----------------------------------"
        if beam_timing > time_cut:
            E_out.Fill(muonE)
            if minos_match == 0:
                E_out_nomatch.Fill(muonE)
            elif minos_match == 1:
                E_out_match.Fill(muonE)

        else:
            E_in.Fill(muonE)
            if minos_match == 0:
                E_in_nomatch.Fill(muonE)
            elif minos_match == 1:
                E_in_match.Fill(muonE)
                
        # if minos_match == 0:
        #     if beam_timing >= time_cut:
        #         # ETheta_out_nomatch.Fill(muonE, muonTheta)
        #         # ETheta_out.Fill(muonE, muonTheta)
        #         E_out_nomatch.Fill(muonE)
        #         E_out.Fill(muonE)
        #     else:
        #         # ETheta_in_nomatch.Fill(muonE, muonTheta)
        #         # ETheta_in.Fill(muonE, muonTheta)
        #         E_in_nomatch.Fill(muonE)
        #         E_in.Fill(muonE)
        # else:
        #     if beam_timing >= time_cut:
        #         # ETheta_out_match.Fill(muonE, muonTheta)
        #         # ETheta_out.Fill(muonE, muonTheta)
        #         E_out_match.Fill(muonE)
        #         E_out.Fill(muonE)
        #     else:
        #         # ETheta_in_match.Fill(muonE, muonTheta)
        #         # ETheta_in.Fill(muonE, muonTheta)
        #         E_in_match.Fill(muonE)
        #         E_in.Fill(muonE)
        "-----------------------------------"
    "=========================================================="

    """ Energy Canvas """
    "=========================================================="
    if plot_individual == False:
        CanvasE = ROOT.TCanvas(f"MuonEnergy_{time_cut}ns", f"Muon{time_cut}ns")
        CanvasE.Divide(3,1)

        # In Time Graphs
        CanvasE.cd(1)
        E_in.Scale(1/E_in.Integral(), 'nosw2')
        E_out.Scale(1/E_out.Integral(), 'nosw2')
        E_out.Draw()
        E_in.Draw('same')
    
        CanvasE.cd(2)
        E_in_match.Scale(1/E_in_match.Integral(), 'nosw2')
        E_out_match.Scale(1/E_out_match.Integral(), 'nosw2')
    
        E_out_match.Draw()
        E_in_match.Draw('same')
    
        CanvasE.cd(3)
        E_in_nomatch.Scale(1/E_in_nomatch.Integral(), 'nosw2')
        E_out_nomatch.Scale(1/E_out_nomatch.Integral(), 'nosw2')
    
        E_out_nomatch.Draw()
        E_in_nomatch.Draw('same')

        CanvasE.Update()
        "=========================================================="

        """ Energy Angle Canvas """
        "=========================================================="
        # CanvasET = ROOT.TCanvas(f"MuonEnergyAngle_{time_cut}ns", f"Muon{time_cut}ns")
        # CanvasET.Divide(3,2)

        # # In Time Graphs
        # CanvasET.cd(1)
        # ETheta_in.Draw('col')
        # CanvasET.cd(2)
        # ETheta_in_match.Draw('col')
        # CanvasET.cd(3)
        # ETheta_in_nomatch.Draw('col')

        # # Out of Time Graphs
        # CanvasET.cd(4)
        # ETheta_out.Draw('col')
        # CanvasET.cd(5)
        # ETheta_out_match.Draw('col')
        # CanvasET.cd(6)
        # ETheta_out_nomatch.Draw('col')
        # CanvasET.Update()
        "=========================================================="
        # file.WriteObject(CanvasET, f"MuonEnergyAngle_{time_cut}ns")
        file.WriteObject(CanvasE, f"MuonEnergy_{time_cut}ns")
        
    elif plot_individual == True:
        # file.WriteObject(ETheta_in, f"MuonEnergyAngle_in{time_cut}ns")
        # file.WriteObject(ETheta_out, f"MuonEnergyAngle_out{time_cut}ns")
        # file.WriteObject(ETheta_in_match, f"MuonEnergyAngle_in_match{time_cut}ns")
        # file.WriteObject(ETheta_in_match, f"MuonEnergyAngle_out_match{time_cut}ns")
        # file.WriteObject(ETheta_in_nomatch, f"MuonEnergyAngle_in_nomatch{time_cut}ns")
        # file.WriteObject(ETheta_out_nomatch, f"MuonEnergyAngle_out_nomatch{time_cut}ns")

        allCanvas = ROOT.TCanvas(f"MuonEnergy_{time_cut}ns", f"Muon{time_cut}ns")
        allCanvas.cd()
        E_in.Scale(1/E_in.Integral(), "nosw2")
        E_out.Scale(1/E_out.Integral(), "nosw2")

        E_out.Draw()
        E_in.Draw('same')
        allCanvas.Update()

        # matchCanvas = ROOT.TCanvas(f"MuonEnergy_match{time_cut}ns", f"Muon{time_cut}ns")
        # matchCanvas.cd()
        # E_in_match.Scale(1/E_in_match.Integral(), "nosw2")
        # E_out_match.Scale(1/E_out_match.Integral(), "nosw2")

        # E_in_match.Draw()
        # E_out_match.Draw('same')
        # allCanvas.Update()
        
        # nomatchCanvas = ROOT.TCanvas(f"MuonEnergy_nomatch{time_cut}ns", f"Muon{time_cut}ns")
        # nomatchCanvas.cd()
        # E_in_nomatch.Scale(1/E_in_nomatch.Integral(), "nosw2")
        # E_out_nomatch.Scale(1/E_out_nomatch.Integral(), "nosw2")

        # E_out_nomatch.Draw()
        # E_in_nomatch.Draw('same')
        # allCanvas.Update()

        file.WriteObject(allCanvas, allCanvas.GetName())
        # file.WriteObject(nomatchCanvas, nomatchCanvas.GetName())
        
        # Es = [ (E_in, E_out, ""), (E_in_match, E_out_match, "match"), (E_in_nomatch, E_out_nomatch, "nomatch") ]
        # for Eset in Es:
        #     Canvas = ROOT.TCanvas(f"MuonEnergy_{Eset[2]}{time_cut}ns", f"Muon{time_cut}ns")
        #     Canvas.cd()
        #     # Eset[0].Scale(1/Eset[0].Integral(), "nosw2")
        #     # Eset[1].Scale(1/Eset[1].Integral(), "nosw2")
            
        #     Eset[1].Draw()
        #     Eset[0].Draw('same')
        #     file.WriteObject(Canvas, Canvas.GetName())

    file.Close()

In [7]:
def CreateMuonProtonHists(filename, timing_cut, transverse_energy_cut, hist_bins = 75):
    file = ROOT.TFile(f"{filename}.root", 'update')
    tree = file.Get("Analysis")
    
    tree.SetBranchStatus("*", False)
    branches = ("muon_proton_P_T", "MasterAnaDev_muon_E", "MasterAnaDev_proton_T_fromdEdx", "MasterAnaDev_pion_P", "beam_offset")
    for branch in branches:
        tree.SetBranchStatus(branch, True)

    """ Creating Histograms """
    "=========================================================="
    EHist = ROOT.THStack("MPEnergy", "Muon + Proton Energy [MeV]")#, 0, 2000, 100)
    
    EHist_in = ROOT.THStack("MPEnergy_in", "In Time mu + p E [MeV]")#, 0, 2000, 100) 
    EHist_lowP_in = ROOT.TH1D("Energy_lowPTin", "In Time mu + p E, Low P_T [MeV]", hist_bins, 0, 1000)
    EHist_bigP_in = ROOT.TH1D("Energy_highPTin", "In Time mu + p E, High P_T [MeV]", hist_bins, 0, 1000)
    EHist_lowP_in.SetLineColor(1)
    EHist_bigP_in.SetLineColor(13)

    EHist_out = ROOT.THStack("MPEnergy_out", "Out of Time mu + p E [MeV]")#, 0, 2000, 100) 
    EHist_lowP_out = ROOT.TH1D("Energy_lowPTout", "Out of Time mu + p E, Low P_T [MeV]", hist_bins, 0, 1000)
    EHist_bigP_out = ROOT.TH1D("Energy_highPTout", "Out of Time mu + p E, High P_T [MeV]", hist_bins, 0, 1000)
    EHist_lowP_out.SetLineColor(2)
    EHist_bigP_out.SetLineColor(46)
    
    # PTHist = ROOT.TH1D("MuonProtonTransverseMom", "Muon + Proton Transverse Momentum", 0, 1000, 100)
    "=========================================================="
    
    """ Event Loop """
    "=========================================================="
    for event in tree:
        transverse_mom = event.muon_proton_P_T
        muonE = event.MasterAnaDev_muon_E
        protonT = event.MasterAnaDev_proton_T_fromdEdx
        E = muonE + protonT
        
        if transverse_mom < 0:
            continue
        elif transverse_mom < transverse_energy_cut:
            if np.abs(event.beam_offset) < timing_cut:
                EHist_lowP_in.Fill(E)
            else:
                EHist_lowP_out.Fill(E)
        else:
            if np.abs(event.beam_offset) < timing_cut:
                EHist_bigP_in.Fill(E)
            else:
                EHist_bigP_out.Fill(E)
    "=========================================================="

    """ Combining Histograms """
    "=========================================================="
    EHist.Add(EHist_bigP_out)
    EHist.Add(EHist_lowP_out)
    EHist.Add(EHist_bigP_in)
    EHist.Add(EHist_lowP_in)

    EHist_in.Add(EHist_lowP_in)
    EHist_in.Add(EHist_bigP_in)
    
    EHist_out.Add(EHist_lowP_out)
    EHist_out.Add(EHist_bigP_out)

    inoutCanvas = ROOT.TCanvas("MPEnergyHists_Separated", "Muon Proton Energy [MeV]")
    inoutCanvas.Divide(2,1)
    inoutCanvas.cd(1)
    EHist_in.Draw()
    inoutCanvas.cd(2)
    EHist_out.Draw()
    inoutCanvas.Update()

    file.WriteObject(inoutCanvas, "MPEnergyHists_Separated")
    file.WriteObject(EHist, "MPEnergyHists")
    file.Close()

# Testing Area

In [94]:
FileAnalysis("check", "parallelization_test", 1, 75.24572772497888)

Beginning Analysis on file set 1
--------------------------------------------
Data tree created
Analysis branches created
Branch addresses set


In [8]:
CreateMuonProtonHists("p1_ana_cond", 6, 250, hist_bins = 100)

# Old Files

In [55]:
def plot_trans_momentum_CondenseAnalysis(output_filename):
    AnaChain = ROOT.TChain("Analysis", "AnaChain")
    
    for (root, dirs, files) in os.walk("./"):
        for file in files:
            if output_filename in file:
                AnaChain.Add(f"{root}/{file}")

    output_file = ROOT.TFile(f"{output_filename}_cond.root", 'recreate')
    
    output_tree = AnaChain.CloneTree(0)
    output_tree.CopyEntries(AnaChain)
    output_tree.SetName("Analysis")
    output_tree.SetTitle("Minerva Analysis")
    
    """ Analysis Histograms """
    "=========================================================="
    track_time_hstack = ROOT.THStack("TrackTimeHist", ";Track Time [ns]")
    track_time_hin = ROOT.TH1D("TrackTimeHist_in", ";Track Time [ns]", 10000, 0., 10000.)
    track_time_hout = ROOT.TH1D("TrackTimeHist_out", ";Track Time [ns]", 10000, 0., 10000.)
    track_time_hin.SetLineColor(1)
    track_time_hout.SetLineColor(2)

    beam_period = 18.831
    beam_size = beam_period/2 + 0.00001
    
    offset_hstack = ROOT.THStack("OffsetHist", ";Beam Offset [ns]")
    offset_hin = ROOT.TH1D( "OffsetHist_in", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_hout = ROOT.TH1D( "OffsetHist_out", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_htotal = ROOT.TH1D( "OffsetHist_Total", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_hin.SetLineColor(1)
    offset_hout.SetLineColor(2)

    trans_mom_hstack = ROOT.THStack("TransferredMom", ";Transferred 4 Momentum |t| [MeV^2]")
    trans_mom_hin = ROOT.TH1D("TransferredMom_in", ";Transferred 4 Momentum |t| [MeV^2]", 75, 0, 5.*10**6)
    trans_mom_hout = ROOT.TH1D("TransferredMom_out", ";Transferred 4 Momentum |t| [MeV^2]", 50, 0, 5.*10**6)
    trans_mom_hin.SetLineColor(1)
    trans_mom_hout.SetLineColor(2)
    
    print("Histograms Created")
    "=========================================================="

    """ Filling Histograms """
    "=========================================================="
    for event in AnaChain:
        offset_htotal.Fill(event.beam_offset)
        
        if event.beam_timing == 0:
            track_time_hout.Fill(event.track_time)
            offset_hout.Fill(event.beam_offset)
            trans_mom_hout.Fill(event.trans_four_mom)
        elif event.beam_timing == 1:
            track_time_hin.Fill(event.track_time)
            offset_hin.Fill(event.beam_offset)
            trans_mom_hin.Fill(event.trans_four_mom)
    "=========================================================="

    """ Histogram Drawing """
    "=========================================================="
    # Track Time
    track_time_hstack.Add(track_time_hout)
    track_time_hstack.Add(track_time_hin)
    "-----------------------------"
    
    """ Beam Offset """
    "-----------------------------"
    offset_hstack.Add(offset_hout)
    offset_hstack.Add(offset_hin)
    
    oCanvas = ROOT.TCanvas("BeamOffsetHist", "BeamOffsetHist")
    oCanvas.cd()
    offset_hstack.Draw()
    oCanvas.Update()
    
    # Defining function for a gaussian on top of a uniform background
    gaus_bg = ROOT.TF1("gaus_bg", "gaus(0) + [3]", -9.5,9.5) 
    gaus_bg.SetParName(3, "background")
    
    gaus_bg.SetParameters(2000, 0, 2, 20)

    # Fitting Packet Offset 
    hist_fit = offset_htotal.Fit("gaus_bg", "S Quiet", "", -9.5, 9.5)
    hist_fit.Draw()
    oCanvas.Update()
    "-----------------------------"
    
    """ Transferred 4 Momentum """
    "-----------------------------"
    tmCanvas = ROOT.TCanvas("TransMomentumHist", "TransMomentum")
    tmCanvas.cd()

    trans_mom_hin.Scale(1/trans_mom_hin.Integral(), 'width nosw2')
    trans_mom_hout.Scale(1/trans_mom_hout.Integral(), 'width nosw2')
    
    trans_mom_hout.Draw()
    trans_mom_hin.Draw('same')
    
    tmCanvas.Update()
    "-----------------------------"
    "=========================================================="

    output_file.WriteObject(output_tree, output_tree.GetName())
    output_file.WriteObject(track_time_hstack, "TrackTimeHist")
    output_file.WriteObject(oCanvas, "BeamOffsetHist")
    output_file.WriteObject(tmCanvas, "TransferredMomentumHist")
    output_file.Close()

## Precreating and saving individual bins for track time

In [81]:
def TrackTimeStep(histogram, scan_start, int_cut, chi2_cut):
    nbins = histogram.GetNbinsX()           # number of bins in the histogram
    tmax  = histogram.GetXaxis().GetXmax()  # maximum time value
    bin_density = nbins/tmax                # density of the bins ( dbins / dt )
    
    fit_radius  = 6                              # radius about the mean beam time in ns to fit a gaussian curve
    beam_radius = round(fit_radius*bin_density)  # radius aobut the mean beam time in bins
    
    beam_period      = 18.831                          # beam periodicity in ns
    beam_period_bins = round(beam_period*bin_density)  # beam periodicity in bins  

    mean_times = []  # list to store times of each mean time
    
    # Progress Counters
    total_beam_count = 0
    small_beam_count = 0
    bad_fit_count    = 0
    good_fit_count   = 0
    
    
    nbeams = int(nbins/beam_period_bins)       # Number of beam packets based on number of bins and period between beams
    curr_bin  = round(scan_start*bin_density)  # current bin for the search
    
    for beam in range(nbeams):
        total_beam_count += 1

        beam_int = histogram.Integral( curr_bin - beam_radius, curr_bin + beam_radius )

        if beam_int < int_cut: # Removing beam packets with few or no events
            mean_times.append(None)
            small_beam_count += 1
        else:
            curr_time = round(curr_bin/bin_density) # current time to fit the gaussian around
            fit = histogram.Fit("gaus", "S Quiet", "", curr_time-fit_radius, curr_time+fit_radius)
            mean  = fit.Parameter(1)
            chi2  = fit.Chi2()
            if chi2 > chi2_cut:
                mean_times.append(None)
                bad_fit_count += 1
            else:
                mean_times.append(mean)
                good_fit_count += 1

        # Setting the bin for the next beam based on the peak of the current beam
        curr_bin = round(mean*bin_density) + beam_period_bins
        
    print(f"Total of {total_beam_count} beam packets")
    print(f"{small_beam_count} Packets rejected for having too little data")
    print(f"{bad_fit_count} Packets rejected for having poor fits")
    print(f"{good_fit_count} Packets accepted")

    return mean_times

In [6]:
def FileAnalysisOld(output_filename, fileset_name, set_number, beam_start_time):
    print(f"Beginning Analysis on file set {set_number}")
    print("--------------------------------------------")
    # Importing Data
    output_file = ROOT.TFile(f"{output_filename}{set_number}.root", 'recreate')
    data_tree = CreateDataTree(fileset_name, set_number)
    print("Data tree created")

    """ Defining Analysis Branches """
    "=========================================================="
    ana_tree = data_tree.CloneTree(0)
    
    track_time         = np.array([0.]) # event track time [ns]
    beam_offset        = np.array([0.]) # event offset from beam center [ns]
    transferred_mom    = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) # transferred 4mom |t| in muon-pion interactions [MeV^2]

    ana_tree.Branch("track_time", track_time, "tracktime/D")
    ana_tree.Branch("beam_offset", beam_offset, "beam_offset/D")
    ana_tree.Branch("trans_four_mom", transferred_mom, "trans_four_mom/D")
    
    print("Analysis branches created")
    "=========================================================="

    """ Event Loop """
    "=========================================================="
    print("Beginning Event Loop")
    # Track Time Variables
    beam_period = 18.831
    beam_size = beam_period/2 + 0.00001 # Adding a very small overlap to avoid the possibility of getting stuck in the beam offset loop, better way to prevent this? 
    
    prev_gate = 0
    prev_time = 0.
    for event in data_tree:
        """ Event Timing """
        "-----------------------------"
        vtx_time = event.muon_trackVertexTime - (event.vtx[2] - 4000)/300.
        if event.ev_gate == prev_gate:
            time = vtx_time - prev_time
            # If the event track time is smaller than the first beam boundaries, remove it
            if time < beam_start_time - beam_size:
                track_time[0] = -1111
                beam_offset[0] = -1111
                # beam_timing[0] = -1111
            # Cutting out the weird peak at ~348 ns
            elif time > 344 and time < 352:
                track_time[0] = -8888
                beam_offset[0] = -8888
                # beam_timing[0] = -8888
            else:
                track_time[0] = time # Filling the track time before calculating the beam offset
                time -= beam_start_time # Adjusting for the overall beam shift
                # Subtracting by intervals of the beam period until it falls within the expected beam width
                while np.abs(time) > beam_size:
                    time -= beam_period 

                """ REVISE HOW TO STORE TIMECUT INFORMATION """
                beam_offset[0] = time
                # if   np.abs(time) >= 7.0:
                #     beam_timing[0] = 7.0
                # elif np.abs(time) >= 6.5:
                #     beam_timing[0] = 6.5
                # elif np.abs(time) >= 6.0:
                #     beam_timing[0] = 6.0
                # elif np.abs(time) >= 5.5:
                #     beam_timing[0] = 5.5
                # elif np.abs(time) >= 5.0:
                #     beam_timing[0] = 5.0
                # else:
                #     beam_timing[0] = 0
        else:
            track_time[0]  = -9999
            beam_offset[0] = -9999
            # beam_timing[0] = -9999

        prev_gate = event.ev_gate
        prev_time = vtx_time
        "-----------------------------"
        
        """ Transferred Four Momentum """
        "-----------------------------"
        # Params determining a good muon detection
        minos_match = event.n_minos_matches 
        minos_track = event.MasterAnaDev_minos_trk_is_ok
        
        if (minos_track == 1) and (minos_match == 1):
            muonE = event.MasterAnaDev_muon_E
            muonP = event.MasterAnaDev_muon_P
            muonTheta = event.MasterAnaDev_muon_theta

            # Finding Transverse and Longitudinal muon momentum
            muonP_T = muonP * np.sin(muonTheta)
            muonP_L = muonP * np.cos(muonTheta)

            # Looping through pion array
            for i, pionE in enumerate(event.MasterAnaDev_pion_E):
                # Flagging invalid pions
                if pionE < 0:
                    transferred_mom[i] = -9999
                else:
                    pionP = event.MasterAnaDev_pion_P[i]
                    pionTheta = event.MasterAnaDev_pion_theta[i]

                    pionP_T = pionP * np.sin(pionTheta)
                    pionP_L = pionP * np.cos(pionTheta)

                    transferred_mom[i] = (pionE + muonE - pionP_L - muonP_L)**2 + (pionP_T + muonP_T)**2
            
        else:
            for i in range(len(transferred_mom)):
                transferred_mom[i] = -9999
        "-----------------------------"
        ana_tree.Fill()
    print("Event Loop Completed")
    "=========================================================="
    
    """ Writing Outputs """
    "=========================================================="
    ana_tree.SetName("Analysis")
    ana_tree.SetTitle("Minerva Analysis")
    output_file.WriteObject(ana_tree, "Analysis")    
    print("File Written")
    "=========================================================="

In [ ]:
def TrackTimeEach(histogram, scan_start, int_cut, chi2_cut):
    nbins = histogram.GetNbinsX()           # number of bins in the histogram
    tmax  = histogram.GetXaxis().GetXmax()  # maximum time value
    bin_density = nbins/tmax                # density of the bins ( dbins / dt )
    
    fit_radius  = 6                              # radius about the mean beam time in ns to fit a gaussian curve
    beam_radius = round(fit_radius*bin_density)  # radius aobut the mean beam time in bins
    
    beam_period      = 18.831                          # beam periodicity in ns
    beam_period_bins = round(beam_period*bin_density)  # beam periodicity in bins  

    mean_times = []  # list to store times of each mean time
    
    # Progress Counters
    total_beam_count = 0
    small_beam_count = 0
    bad_fit_count    = 0
    good_fit_count   = 0
    
    
    nbeams = int(nbins/beam_period_bins)       # Number of beam packets based on number of bins and period between beams
    curr_bin  = round(scan_start*bin_density)  # current bin for the search
    
    for beam in range(nbeams):
        total_beam_count += 1

        beam_int = histogram.Integral( curr_bin - beam_radius, curr_bin + beam_radius )

        if beam_int < int_cut: # Removing beam packets with few or no events
            mean_times.append(None)
            small_beam_count += 1
        else:
            curr_time = round(curr_bin/bin_density) # current time to fit the gaussian around
            fit = histogram.Fit("gaus", "S Quiet", "", curr_time-fit_radius, curr_time+fit_radius)
            mean  = fit.Parameter(1)
            chi2  = fit.Chi2()
            if chi2 > chi2_cut:
                mean_times.append(None)
                bad_fit_count += 1
            else:
                mean_times.append(mean)
                good_fit_count += 1

        # Setting the bin for the next beam based on the peak of the current beam
        curr_bin = round(mean*bin_density) + beam_period_bins
        
    print(f"Total of {total_beam_count} beam packets")
    print(f"{small_beam_count} Packets rejected for having too little data")
    print(f"{bad_fit_count} Packets rejected for having poor fits")
    print(f"{good_fit_count} Packets accepted")

    return mean_times

In [ ]:
def genTrackTimeAll(source_dir, output_filename, scan_start, beam_size = 5, int_cut= 750, chi2_cut = 50, hist_bins = 40000):
    # Creating TChain from all files in source directory
    Chain = ROOT.TChain( "MasterAnaDev", "Chain" )
    nfiles = 0 

    for (root, dirs, files) in os.walk(f"{source_dir}/"):
        for file in files:
            if ("MasterAnaDev_data_AnaTuple_run" in file) and ("Playlist.root" in file):
                Chain.Add(f"{root}/{file}")
                nfiles += 1

    print(f"Chain created with {Chain.GetEntries()} entries from {nfiles} files.")

    # Setting only the relevant branches as active
    Chain.SetBranchStatus("*", False)
    Chain.SetBranchStatus("ev_gate", True)
    Chain.SetBranchStatus("vtx", True)
    Chain.SetBranchStatus("muon_trackVertexTime", True)
    
    # Histogram to which we can fit the beam structure
    tracktime_hist = ROOT.TH1D( "Track_Time_Hist", ";Track Time (ns)", hist_bins, 0., 10000. )

    # Not sure what these are for...
    prev_gate = 0
    prev_time = 0.

    # Calculating track time for each event
    for entry in Chain:        
        vtx_time = entry.muon_trackVertexTime - (entry.vtx[2] - 4000)/300.
        if entry.ev_gate == prev_gate:
            tracktime = vtx_time - prev_time
            tracktime_hist.Fill(tracktime)

        prev_gate = entry.ev_gate
        prev_time = vtx_time

    beam_times = TrackTimeEach(tracktime_hist, scan_start, int_cut, chi2_cut)

    output_file = open(f"{output_filename}_each.txt", "w", newline = '')
    ofile = csv.writer(output_file)
        
    # ofile.writerow( ["mean time", "lower time", "upper time"] )
        
    for time in beam_times:
        ofile.writerow( [time, time-beam_size, time+beam_size] )
    output_file.close()

In [ ]:
def getTTSFile(filename):
    file = open(filename, 'r')
    csvfile = csv.reader(file)
    data = []
    for line in csvfile:
        datum = []
        for l in line:
            datum.append(float(l))
        data.append(datum)
    return data

## Old old files

In [ ]:
branch_list = {'ev_gate',
               'vtx',
               'muon_trackVertexTime', 
               'MasterAnaDev_muon_E',
               'MasterAnaDev_muon_P',
               'MasterAnaDev_muon_theta', 
               'MasterAnaDev_minos_trk_is_ok', 
               'n_minos_matches', 
               'MasterAnaDev_in_fiducial_area',
               'MasterAnaDev_pion_E',
               'MasterAnaDev_pion_P',
               'MasterAnaDev_pion_theta',  
               'recoil_energy_nonmuon_vtx200mm'
              } 

In [ ]:
def ImportData(source_dir, filename, prop_list):
    OutputFile = ROOT.TFile(f"{filename}.root", "recreate")

    # Creating a TChain for all the files in the analysis
    Chain = ROOT.TChain( "MasterAnaDev", "Chain" )
    nfiles = 0 # Progress keeping

    # Looping through all the files in the source directory and any subfolders
    for (root, dirs, files) in os.walk(f"{source_dir}/"):
        for file in files:
            # To avoid adding in non-relevant files, only adding file if it has key strings
            if ("MasterAnaDev_data_AnaTuple_run" in file) and ("Playlist.root" in file):
                Chain.Add(f"{root}/{file}")
                nfiles += 1
                    
    # Printing total number of files and entries 
    print(f"Chain created with {Chain.GetEntries()} entries from {nfiles} files.")

    # To speed up processing, inactivating irrelevant branches
    Chain.SetBranchStatus("*",False)

    # Reactivating relevant branches
    for prop in prop_list:
        Chain.SetBranchStatus(prop, True)

    # Copying active data into a new file so it's easier to manipulate
    OutputTree = Chain.CloneTree(0)
    OutputTree.CopyEntries(Chain)
    OutputTree.SetName("SourceData")
    OutputTree.SetTitle("Source Data")
    
    OutputFile.WriteObject(OutputTree, OutputTree.GetName())

In [ ]:
def TrackTimeScan(histogram, scan_start, int_cut, chi2_cut):
    nbins = histogram.GetNbinsX()           # number of bins in the histogram
    tmax  = histogram.GetXaxis().GetXmax()  # maximum time value
    bin_density = nbins/tmax                # density of the bins ( dbins / dt )
    
    fit_radius  = 6                              # radius about the mean beam time in ns to fit a gaussian curve
    beam_radius = round(fit_radius*bin_density)  # radius aobut the mean beam time in bins
    
    beam_period      = 18.8                            # beam periodicity in ns
    beam_period_bins = round(beam_period*bin_density)  # beam periodicity in bins  

    mean_times = []  # list to store times of each mean time
    # sigmas     = []  # list to store the sigma for the fit of each packet distribution
    # chi2_list  = []  # list to store chi2 value for the fit of each packet
    
    # Progress Counters
    total_beam_count = 0
    small_beam_count = 0
    bad_fit_count    = 0
    good_fit_count   = 0
    
    
    nbeams = int(nbins/beam_period_bins)       # Number of beam packets based on number of bins and period between beams
    curr_bin  = round(scan_start*bin_density)  # current bin for the search
    
    for beam in range(nbeams):
        total_beam_count += 1

        beam_int = histogram.Integral( curr_bin - beam_radius, curr_bin + beam_radius )

        if beam_int < int_cut: # Removing beam packets with few or no events
            mean_times.append(None)
            # sigmas.append(None)
            # chi2_list.append(None)
            small_beam_count += 1
        else:
            curr_time = round(curr_bin/bin_density) # current time to fit the gaussian around
            fit = histogram.Fit("gaus", "S Quiet", "", curr_time-fit_radius, curr_time+fit_radius)
            mean  = fit.Parameter(1)
            #sigma = fit.Parameter(2)
            chi2  = fit.Chi2()
            if chi2 > chi2_cut:
                mean_times.append(None)
                # sigmas.append(None)
                # chi2_list.append(None)
                bad_fit_count += 1
            else:
                mean_times.append(mean)
                # sigmas.append(sigma)
                # chi2_list.append(chi2)
                good_fit_count += 1

        # Setting the bin for the next beam based on the peak of the current beam
        curr_bin = round(mean*bin_density) + beam_period_bins
        
    print(f"Total of {total_beam_count} beam packets")
    print(f"{small_beam_count} Packets rejected for having too little data")
    print(f"{bad_fit_count} Packets rejected for having poor fits")
    print(f"{good_fit_count} Packets accepted")

    # Finding the period between each beam
    # mean_diff = []
    # for i,time in enumerate(mean_times):
    #     if i == 0:
    #         continue
            
    #     t_prev = mean_times[i-1]
        
    #     if (t_prev == None) or (time == None):
    #         continue
    #     else:
    #         mean_diff.append(time-t_prev)

    return mean_times#, sigmas, chi2_list#, mean_diff

In [ ]:
def getTrackTimeStructure(data_tree, scan_start, int_cut, chi2_cut, beam_size):
    # file = ROOT.TFile(f"{filename}.root", "read")
    # data_tree = file.Get("SourceData")
    
    tracktime_hist = ROOT.TH1D( "Track_Time_Hist", ";Track Time (ns)", 40000, 0., 10000. )   # Plotting track time events as a high rez histogram to resolve time structure

    # Not sure what these are for...
    prev_gate = 0
    prev_time = 0.

    # Calculating track time for each event
    for entry in data_tree:        
        vtx_time = entry.muon_trackVertexTime - (entry.vtx[2] - 4000)/300.
        if entry.ev_gate == prev_gate:
            tracktime = vtx_time - prev_time
            tracktime_hist.Fill(tracktime)

        prev_gate = entry.ev_gate
        prev_time = vtx_time

    beam_times, beam_sigmas, beam_chi2s, beam_periods = TrackTimeScan(tracktime_hist, scan_start, int_cut, chi2_cut)
    
    beam_struc_list = []
    for time in beam_times:
        if time != None:
            beam_struc_list.append( (time, time-beam_size, time+beam_size) )

    return beam_struc_list, beam_sigmas, beam_chi2s, beam_periods

In [ ]:
def oldFileAnalysis(filename, beam_start_time = 75, beam_int_cut = 750, beam_chi2_cut = 50, return_beam_details = False, beam_size = 9.5, beam_time_cut = 5):
    # Importing Data
    file = ROOT.TFile(f"{filename}.root", 'update')
    data_tree = file.Get("SourceData")

    """ Defining Analysis Branches """
    "=========================================================="
    ana_tree = ROOT.TTree("Analysis", "Analysis Properties")

    track_time         = np.array([0.]) # event track time [ns]
    beam_offset        = np.array([0.]) # event offset from beam center [ns]
    beam_timing        = np.array([0 ]) # flagging in vs out of time [0: out, 1: in]
    muon_Etheta2       = np.array([0.]) # muon Energy*Theta^2
    transferred_mom    = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) # transferred 4mom |t| in muon-pion interactions [MeV^2]
    coherent_pion_flag = np.array([0 ]) # flagging coherent pion production events [0: not an event, 1: possible event]

    ana_tree.Branch("track_time", track_time, "tracktime/D")
    ana_tree.Branch("beam_offset", beam_offset, "beam_offset/D")
    ana_tree.Branch("beam_timing", beam_timing, "beam_timing/I")
    ana_tree.Branch("muon_Etheta2", muon_Etheta2, "muon_Etheta2/D")
    ana_tree.Branch("trans_four_mom", transferred_mom, "trans_four_mom/D")
    ana_tree.Branch("coherent_pion_flag", coherent_pion_flag, "coherent_pion_flag/O")
    
    print("Branches Created")
    "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"

    """ Analysis Histograms """
    "=========================================================="
    track_time_hstack = ROOT.THStack("TrackTimeHist", ";Track Time [ns]")
    track_time_hin = ROOT.TH1D("TrackTimeHist_in", ";Track Time [ns]", 10000, 0., 10000.)
    track_time_hout = ROOT.TH1D("TrackTimeHist_out", ";Track Time [ns]", 10000, 0., 10000.)
    track_time_hin.SetLineColor(1)
    track_time_hout.SetLineColor(2)

    offset_hstack = ROOT.THStack("OffsetHist", ";Beam Offset [ns]")
    offset_hin = ROOT.TH1D( "OffsetHist_in", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_hout = ROOT.TH1D( "OffsetHist_out", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_htotal = ROOT.TH1D( "OffsetHist_Total", ";Offset From Peak [ns]", 250, -(beam_size + 0.5), (beam_size + 0.5) )
    offset_hin.SetLineColor(1)
    offset_hout.SetLineColor(2)

    if return_beam_details == True:
        beam_period_hist = ROOT.TH1D("BeamPeriodHist", ";Period [ns]", 100 , 18., 20.)
        beam_chi2_hist   = ROOT.TH1D( "BeamChi2Hist", ";Beam Chi2", 100, 20, beam_chi2_cut + 1)
        beam_sigma_hist  = ROOT.TH1D( "BeamSigmaHist", ";Beam Sigma", 100 , 1.5, 2.5)

    muon_Etheta_hstack = ROOT.THStack("MuonEtheta", ";Muon E Theta2 [MeV]")
    muon_Etheta_hin = ROOT.TH1D("MuonEtheta_in", ";Muon E Theta2 [MeV]", 200, 0, 900)
    muon_Etheta_hout = ROOT.TH1D("MuonEtheta_out", ";Muon E Theta2 [MeV]", 100, 0, 900)
    muon_Etheta_hin.SetLineColor(1)
    muon_Etheta_hout.SetLineColor(2)

    trans_mom_hstack = ROOT.THStack("TransferredMom", ";Transferred 4 Momentum |t| [MeV^2]")
    trans_mom_hin = ROOT.TH1D("TransferredMom_in", ";Transferred 4 Momentum |t| [MeV^2]", 200, 0, 5.*10**6)
    trans_mom_hout = ROOT.TH1D("TransferredMom_out", ";Transferred 4 Momentum |t| [MeV^2]", 100, 0, 5.*10**6)
    trans_mom_hin.SetLineColor(1)
    trans_mom_hout.SetLineColor(2)
    
    print("Histograms Created")
    "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"

    """ Event Loop """
    "=========================================================="
    # Track Time Variables
    prev_gate = 0
    prev_time = 0.

    # Generating Beam Structure
    beam_structure, beam_sigmas, beam_chi2s, beam_periods = getTrackTimeStructure(data_tree, beam_start_time, beam_int_cut, beam_chi2_cut, beam_size)

    print("Beam Structure Created")
    # Beginning Event Loop
    print("Beginning Event Loop")
    for event in data_tree:
        """ Event Timing """
        "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        vtx_time = event.muon_trackVertexTime - (event.vtx[2] - 4000)/300.
        
        if event.ev_gate == prev_gate:
            time = vtx_time - prev_time
            track_time[0] = time
            # Finding corresponding beam and calculating offset
            for beam in beam_structure:
                beam_mean = beam[0]
                beam_min  = beam[1]
                beam_max  = beam[2]
                # If the event time is within current beam range, find the offset
                if (beam_min < time) and (time < beam_max):
                    beam_offset[0] = time - beam_mean
                    # Determining whether the offset time indicates in or out of time
                    if beam_time_cut < np.abs(beam_offset[0]):
                        beam_timing[0] = 0
                    else:
                        beam_timing[0] = 1
        else:
            track_time[0]  = -9999
            beam_offset[0] = -9999
            beam_timing[0] = -9999

        prev_gate = event.ev_gate
        prev_time = vtx_time
        "-----------------------------"
        
        """ Muon Energy-Angle and Transferred Four Momentum """
        "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        # Params determining a good muon detection
        minos_match = event.n_minos_matches 
        minos_track = event.MasterAnaDev_minos_trk_is_ok
        
        if (minos_track == 1) and (minos_match == 1):
            muonE = event.MasterAnaDev_muon_E
            muonP = event.MasterAnaDev_muon_P
            muonTheta = event.MasterAnaDev_muon_theta

            # Muon Energy Angle
            muon_Etheta2[0] = muonE * muonTheta**2

            # Finding Transverse and Longitudinal muon momentum
            muonP_T = muonP * np.sin(muonTheta)
            muonP_L = muonP * np.cos(muonTheta)

            # Looping through pion array
            for i, pionE in enumerate(event.MasterAnaDev_pion_E):
                # Flagging invalid pions
                if pionE < 0:
                    transferred_mom[i] = -9999
                else:
                    pionP = event.MasterAnaDev_pion_P[i]
                    pionTheta = event.MasterAnaDev_pion_theta[i]

                    pionP_T = pionP * np.sin(pionTheta)
                    pionP_L = pionP * np.cos(pionTheta)

                    transferred_mom[i] = (pionE + muonE - pionP_L - muonP_L)**2 + (pionP_T + muonP_T)**2
            
        else:
            muon_Etheta2[0] = -9999
            for i in range(len(transferred_mom)):
                transferred_mom[i] = -9999
        "-----------------------------"
        
        """ Filling Histograms """
        "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        offset_htotal.Fill(beam_offset)
        
        if beam_timing[0] == 0:
            track_time_hout.Fill(track_time)
            offset_hout.Fill(beam_offset)
            muon_Etheta_hout.Fill(muon_Etheta2)
            for mom in transferred_mom:
                trans_mom_hout.Fill(mom)
        elif beam_timing[0] == 1:
            track_time_hin.Fill(track_time)
            offset_hin.Fill(beam_offset)
            muon_Etheta_hin.Fill(muon_Etheta2)
            for mom in transferred_mom:
                trans_mom_hin.Fill(mom)
        "-----------------------------"

        # Filling Analysis Tree
        ana_tree.Fill()
        
    print("Event Loop Done")
    "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"

    """ Histogram Drawing """
    "=========================================================="
    # Track Time
    track_time_hstack.Add(track_time_hout)
    track_time_hstack.Add(track_time_hin)
    "-----------------------------"
    
    """ Beam Offset """
    "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    offset_hstack.Add(offset_hout)
    offset_hstack.Add(offset_hin)
    
    oCanvas = ROOT.TCanvas("BeamOffsetHist", "BeamOffsetHist")
    oCanvas.cd()
    offset_hstack.Draw()
    oCanvas.Update()
    
    # Defining function for a gaussian on top of a uniform background
    gaus_bg = ROOT.TF1("gaus_bg", "gaus(0) + [3]", -9.5,9.5) 
    gaus_bg.SetParName(3, "background")
    
    gaus_bg.SetParameters(2000, 0, 2, 20)

    # Fitting Packet Offset 
    hist_fit = offset_htotal.Fit("gaus_bg", "S Quiet", "", -9.5, 9.5)
    hist_fit.Draw()
    oCanvas.Update()
    "-----------------------------"
    
    """ Muon Energy-Angle """
    "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    mETCanvas = ROOT.TCanvas("MuonETheta2", "MuonETheta2")
    mETCanvas.cd()
    muon_Etheta_hin.Draw()
    mETCanvas.Update()
    mEToutmax = 1.1*muon_Etheta_hout.GetMaximum()
    mETscale = ROOT.gPad.GetUymax()/mEToutmax
    muon_Etheta_hout.Scale(mETscale, "nosw2")
    muon_Etheta_hout.Draw("same")
    mETCanvas.Update()

    mETaxis = ROOT.TGaxis(ROOT.gPad.GetUxmax(),ROOT.gPad.GetUymin(), ROOT.gPad.GetUxmax(), ROOT.gPad.GetUymax(),0,mEToutmax,510,"+L")
    mETaxis.SetLineColor(2)
    mETaxis.SetLabelColor(2)
    mETaxis.Draw()
    mETCanvas.Update()
    
    # muon_Etheta_hstack.Add(muon_Etheta_hout)
    # muon_Etheta_hstack.Add(muon_Etheta_hin)
    "-----------------------------"
    
    """ Transferred 4 Momentum """
    "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    tmCanvas = ROOT.TCanvas("TransMomentumHist", "TransMomentum")
    tmCanvas.cd()
    trans_mom_hin.Draw()
    tmCanvas.Update()
    tmoutmax = 1.1* trans_mom_hout.GetMaximum()
    tmscale = ROOT.gPad.GetUymax()/tmoutmax
    trans_mom_hout.Scale(tmscale, "nosw2")
    trans_mom_hout.Draw("same")
    tmCanvas.Update()

    tmaxis = ROOT.TGaxis(ROOT.gPad.GetUxmax(),ROOT.gPad.GetUymin(), ROOT.gPad.GetUxmax(), ROOT.gPad.GetUymax(),0,tmoutmax,510,"+L")
    tmaxis.SetLineColor(2)
    tmaxis.SetLabelColor(2)
    tmaxis.Draw()
    tmCanvas.Update()
    
    # trans_mom_hstack.Add(trans_mom_hout)
    # trans_mom_hstack.Add(trans_mom_hin)
    "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    
    """ Writing Outputs """
    "=========================================================="
    ana_tree.AddFriend(data_tree)
    file.WriteObject(ana_tree, ana_tree.GetName())    
    file.WriteObject(track_time_hstack, "TrackTimeHist")
    file.WriteObject(oCanvas, "BeamOffsetHist")
    file.WriteObject(mETCanvas, "MuonETheta2Hist")
    file.WriteObject(tmCanvas, "TransferredMomentumHist")
    # file.WriteObject(muon_Etheta_hist_stack, "MuonEThetaHist")
    # file.WriteObject(trans_mom_hist_stack, "TransferredMomentumHist")
    print("File Written")
    "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"